---
sidebar_label: 0G Compute Network
---


# ChatZeroG

This page will help you get started with 0G Compute Network [chat models](../../concepts/chat_models.mdx). For detailed documentation of all `ChatZeroG` features and configurations, head to the [API reference](https://python.langchain.com/api_reference/zerog/chat_models/langchain_zerog.chat_models.ChatZeroG.html).

The [0G Compute Network](https://0g.ai/) provides decentralized AI inference services with verified computations running in Trusted Execution Environments (TEE). This integration allows you to use 0G's distributed GPU network for AI inference through LangChain.

## Overview
### Integration details

| Class | Package | Local | Serializable | JS support | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatZeroG](https://python.langchain.com/api_reference/zerog/chat_models/langchain_zerog.chat_models.ChatZeroG.html) | [langchain-zerog](https://python.langchain.com/api_reference/zerog/index.html) | ❌ | ✅ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-zerog?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-zerog?style=flat-square&label=%20) |

### Model features
| [Tool calling](../../how_to/tool_calling.ipynb) | [Structured output](../../how_to/structured_output.ipynb) | JSON mode | [Image input](../../how_to/multimodal_inputs.ipynb) | Audio input | Video input | [Token-level streaming](../../how_to/chat_streaming.ipynb) | Native async | [Token usage](../../how_to/chat_token_usage_tracking.ipynb) | [Logprobs](../../how_to/logprobs.ipynb) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ❌ | ❌ | ❌ | ❌ | ✅ | ✅ | ✅ | ❌ |

## Setup

To access 0G Compute Network models, you'll need an Ethereum private key for wallet authentication, OG tokens to fund your account, and the `langchain-zerog` integration package.

### Credentials

You'll need an Ethereum private key and OG tokens. Set the `ZEROG_PRIVATE_KEY` environment variable:


In [ ]:
import getpass
import os

if "ZEROG_PRIVATE_KEY" not in os.environ:
    os.environ["ZEROG_PRIVATE_KEY"] = getpass.getpass(
        "Enter your Ethereum private key: "
    )

### Installation

The LangChain 0G integration lives in the `langchain-zerog` package:


In [ ]:
%pip install -qU langchain-zerog

## Instantiation

Now we can instantiate our model object. Note that you'll need to fund your account before making requests:


In [ ]:
from langchain_zerog import ChatZeroG

llm = ChatZeroG(
    model="llama-3.3-70b-instruct",  # or "deepseek-r1-70b"
    temperature=0.7,
    max_tokens=1000,
    # other params...
)

## Account Management

Before using the model, you need to fund your account with OG tokens:


In [ ]:
# Fund your account (required before first use)
await llm.fund_account("0.1")  # Add 0.1 OG tokens

# Check your balance
balance = await llm.get_balance()
print(f"Available balance: {balance['available']} OG tokens")

## Invocation


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="What is the capital of France?"),
]

# Use async invocation for 0G network
ai_msg = await llm.ainvoke(messages)
print(ai_msg.content)
print(f"Usage: {ai_msg.usage_metadata}")
print(f"Provider: {ai_msg.response_metadata['provider_address']}")

## Streaming

The 0G integration supports token-level streaming for real-time responses:


In [ ]:
async for chunk in llm.astream(messages):
    print(chunk.content, end="", flush=True)

## Tool Calling

The 0G integration supports tool calling for function execution:


In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")
    unit: str = Field(default="celsius", description="Temperature unit")


# Bind tools to the model
llm_with_tools = llm.bind_tools([GetWeather])

# Invoke with a tool-calling request
tool_message = [HumanMessage(content="What's the weather like in San Francisco?")]

ai_msg = await llm_with_tools.ainvoke(tool_message)
print(f"Tool calls: {ai_msg.tool_calls}")
print(f"Content: {ai_msg.content}")

## Structured Output

Use structured output to get responses in a specific format:


In [ ]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: int = Field(description="How funny the joke is, from 1 to 10")


# Create structured output model
structured_llm = llm.with_structured_output(Joke)

# Get a structured response
joke = await structured_llm.ainvoke("Tell me a joke about artificial intelligence")
print(f"Setup: {joke.setup}")
print(f"Punchline: {joke.punchline}")
print(f"Rating: {joke.rating}/10")

## Supported Models

The 0G Compute Network currently supports the following models:

- **llama-3.3-70b-instruct**: State-of-the-art 70B parameter model for general AI tasks (TEE verified)
- **deepseek-r1-70b**: Advanced reasoning model optimized for complex problem solving (TEE verified)

All models run in Trusted Execution Environments (TEE) for verified computation.
